# Sequence Prediction Problem Description

In [2]:
import numpy as np
import pandas as pd

length = 10
sequence = [i/float(length) for i in range(length)]
print(sequence)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [5]:
from pandas import concat
from pandas import DataFrame


df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)
df.head(10)

,0,0
1,0.1,0.0
2,0.2,0.1
3,0.3,0.2
4,0.4,0.3
5,0.5,0.4
6,0.6,0.5
7,0.7,0.6
8,0.8,0.7
9,0.9,0.8


In [6]:
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)

print(X.shape, y.shape)

(9, 1, 1) (9,)


In [7]:
X

array([[[0.1]],

       [[0.2]],

       [[0.3]],

       [[0.4]],

       [[0.5]],

       [[0.6]],

       [[0.7]],

       [[0.8]],

       [[0.9]]])

In [8]:
y

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

# LSTM Model and Varied Batch Size

In [14]:
print(*list(X.shape))

9 1 1


In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.optimizers import Adam

n_batch = len(X)
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [18]:
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()

Epoch 1/1
9/9 [==============================] - 1s 130ms/step - loss: 0.3081
Epoch 1/1
9/9 [==============================] - 0s 849us/step - loss: 0.3047
Epoch 1/1
9/9 [==============================] - 0s 459us/step - loss: 0.3014
Epoch 1/1
9/9 [==============================] - 0s 327us/step - loss: 0.2980
Epoch 1/1
9/9 [==============================] - 0s 507us/step - loss: 0.2947
Epoch 1/1
9/9 [==============================] - 0s 433us/step - loss: 0.2915
Epoch 1/1
9/9 [==============================] - 0s 415us/step - loss: 0.2883
Epoch 1/1
9/9 [==============================] - 0s 405us/step - loss: 0.2851
Epoch 1/1
9/9 [==============================] - 0s 390us/step - loss: 0.2820
Epoch 1/1
9/9 [==============================] - 0s 558us/step - loss: 0.2788
Epoch 1/1
9/9 [==============================] - 0s 553us/step - loss: 0.2757
Epoch 1/1
9/9 [==============================] - 0s 336us/step - loss: 0.2727
Epoch 1/1
9/9 [==============================] - 0s 360us/step -

In [21]:
for i in range(len(X)):
	testX, testy = X[i], y[i]
	testX = testX.reshape(1, 1, 1)
	yhat = model.predict(testX, batch_size=1)
	print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

InvalidArgumentError:  Specified a list with shape [9,1] from a tensor with shape [1,1]
	 [[node lstm_1/TensorArrayUnstack/TensorListFromTensor (defined at /home/villacorta/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_7705]

Function call stack:
keras_scratch_graph


# Solution 1: Online Learning (Batch Size = 1)

In [24]:
length = 10
sequence = [i/float(length) for i in range(length)]

# create X/y pairs
df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)

# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)

X.shape, y.shape

((9, 1, 1), (9,))

In [27]:
n_batch = 1
n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [28]:
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()

Epoch 1/1
9/9 [==============================] - 0s 52ms/step - loss: 0.2576
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.2164
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1809
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1496
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.1220
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.0975
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.0761
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.0577
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.0424
Epoch 1/1
9/9 [==============================] - 0s 3ms/step - loss: 0.0302
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.0213
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.0152
Epoch 1/1
9/9 [==============================] - 0s 4ms/step - loss: 0.0116
Epoch 1/1
9

In [29]:
# online forecast
for i in range(len(X)):
	testX, testy = X[i], y[i]
	testX = testX.reshape(1, 1, 1)
	yhat = model.predict(testX, batch_size=1)
	print('>Expected=%.1f, Predicted=%.1f' % (testy, yhat))

>Expected=0.0, Predicted=0.0
>Expected=0.1, Predicted=0.1
>Expected=0.2, Predicted=0.2
>Expected=0.3, Predicted=0.3
>Expected=0.4, Predicted=0.4
>Expected=0.5, Predicted=0.5
>Expected=0.6, Predicted=0.6
>Expected=0.7, Predicted=0.7
>Expected=0.8, Predicted=0.8


# Solution 2: Batch Forecasting (Batch Size = N)

In [34]:
# create sequence
length = 10
sequence = [i/float(length) for i in range(length)]

# create X/y pairs
df = DataFrame(sequence)
df = concat([df, df.shift(1)], axis=1)
df.dropna(inplace=True)

# convert to LSTM friendly format
values = df.values
X, y = values[:, 0], values[:, 1]
X = X.reshape(len(X), 1, 1)

X.shape, y.shape

((9, 1, 1), (9,))

In [37]:
# configure network
n_batch = len(X)
print(n_batch)

n_epoch = 1000
n_neurons = 10
# design network
model = Sequential()
model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

9


In [38]:
# fit network
for i in range(n_epoch):
	model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
	model.reset_states()

Epoch 1/1
9/9 [==============================] - 0s 48ms/step - loss: 0.2527
Epoch 1/1
9/9 [==============================] - 0s 422us/step - loss: 0.2503
Epoch 1/1
9/9 [==============================] - 0s 265us/step - loss: 0.2479
Epoch 1/1
9/9 [==============================] - 0s 567us/step - loss: 0.2456
Epoch 1/1
9/9 [==============================] - 0s 435us/step - loss: 0.2433
Epoch 1/1
9/9 [==============================] - 0s 319us/step - loss: 0.2409
Epoch 1/1
9/9 [==============================] - 0s 309us/step - loss: 0.2387
Epoch 1/1
9/9 [==============================] - 0s 553us/step - loss: 0.2364
Epoch 1/1
9/9 [==============================] - 0s 364us/step - loss: 0.2341
Epoch 1/1
9/9 [==============================] - 0s 793us/step - loss: 0.2319
Epoch 1/1
9/9 [==============================] - 0s 298us/step - loss: 0.2296
Epoch 1/1
9/9 [==============================] - 0s 657us/step - loss: 0.2274
Epoch 1/1
9/9 [==============================] - 0s 305us/step - 

In [39]:
# batch forecast
yhat = model.predict(X, batch_size=n_batch)
for i in range(len(y)):
	print('>Expected=%.1f, Predicted=%.1f' % (y[i], yhat[i]))

>Expected=0.0, Predicted=0.0
>Expected=0.1, Predicted=0.1
>Expected=0.2, Predicted=0.2
>Expected=0.3, Predicted=0.3
>Expected=0.4, Predicted=0.4
>Expected=0.5, Predicted=0.5
>Expected=0.6, Predicted=0.6
>Expected=0.7, Predicted=0.7
>Expected=0.8, Predicted=0.8
